# Set gmail signature for the company account

Trust me, this is the only working sample I came up from several posts and code snippets.
You need a service account api key token json file with "https://www.googleapis.com/auth/gmail.settings.basic" permission.

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install okta python-dotenv

In [ ]:
from google.oauth2.service_account import Credentials as ServiceAccountCredentials
from googleapiclient.discovery import build

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [ ]:
def list_signature(service, email):
  try:
    result = service.users().settings().sendAs().list(userId=email).execute()
    alias_list = result.get("sendAs", [])
    for alias in alias_list:
      if alias.get("isPrimary"):
        #print(f"Primary: {alias}")
        return alias
  except Exception as e:
    print(e)
  return None

def update_signature(service, display_name, email, sig):
  try:
    send_as_configuration = {
        "displayName": display_name,
        "signature": sig,
    }

    # pylint: disable=E1101
    result = (
        service.users()
        .settings()
        .sendAs()
        .patch(
            userId=email,
            sendAsEmail=email,
            body=send_as_configuration,
        )
        .execute()
    )
    print(f'Updated signature for: {result.get("displayName")}, {result.get("sendAsEmail")}')
    return result.get("signature")
  except Exception as error:
    print(f"An error occurred: {error}")
  return None  

In [ ]:
email = 'foo@bar.com'


# Required scopes for Gmail settings
SCOPES = ['https://www.googleapis.com/auth/gmail.settings.basic']

# Service account credentials with user impersonation
creds = ServiceAccountCredentials.from_service_account_file(
    "google-service-account.json", # Path to your service account key file, replace with your file.
    scopes=SCOPES,
    subject=email  # The email of the user to impersonate
)

service = build("gmail", "v1", credentials=creds)
old_alias = list_signature(service, email)
#print(f"Primary alias:{old_alias}")

In [ ]:
first_name = 'profile.firstName'
last_name = 'profile.lastName'
title = 'profile.title'
dept = 'profile.department'

signature = f'''{first_name}&nbsp;{last_name}<br>{title}<br>{dept}<br>'''
new_signature = update_signature(service, f"{first_name} {last_name}", email, old_alias, signature)